In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python3 -m spacy download pl_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-09 20:39:35.073360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 20:39:36.168132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import requests
from time import sleep
import html
import unicodedata
from bs4 import BeautifulSoup
import spacy
import pl_core_news_sm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 

In [3]:
genre = 'polityka-obronna'
page_number = 2
http_address = f'https://defence24.pl/zobacz-wiecej-polityka-obronna?page={page_number}'

response = requests.get(http_address)
content = response.content.decode('utf-8')
content = html.unescape(content)
content = unicodedata.normalize('NFKD',content)

In [5]:
soup = BeautifulSoup(content, 'html.parser')
a = soup.find_all('a', {'class': 'tile__title stretched-link'})
a[0].get_text().strip()

'Warszawa proponuje nowe podejście do broni atomowej. Berlin zaakceptuje? [KOMENTARZ]'

In [7]:
def get_data(genre, number_of_pages):
    
    labels = []
    all_titles = []


    for page_number in range(1, number_of_pages): # iteracja po numerze strony
            print(f'Getting data about {genre} from page number {page_number}')
            
            http_address = f'https://defence24.pl/{genre}?page={page_number}' # genre - zakładka film
            response = requests.get(http_address)
            content = response.content.decode("utf-8")
            content = html.unescape(content)
            content = unicodedata.normalize('NFKD',content)

            soup = BeautifulSoup(content, 'html.parser')
            a = soup.find_all('a', {'class': 'tile__title stretched-link'})
            titles = [x.get_text().strip() for x in a]

            all_titles.extend(titles)

            
            labels.extend([genre.replace("zobacz-wiecej-","")] * len(titles))
            
            sleep(5)
        
    return {'title': all_titles, 'label':labels}

In [8]:
    websites_catalogs = ["zobacz-wiecej-polityka-obronna","zobacz-wiecej-przemysl","zobacz-wiecej-geopolityka","zobacz-wiecej-sily-zbrojne"]
    lista = []
    for catalogs in websites_catalogs:
     x = get_data(catalogs,10)
     lista.append(x)

Getting data about zobacz-wiecej-polityka-obronna from page number 1
Getting data about zobacz-wiecej-polityka-obronna from page number 2
Getting data about zobacz-wiecej-polityka-obronna from page number 3
Getting data about zobacz-wiecej-polityka-obronna from page number 4
Getting data about zobacz-wiecej-polityka-obronna from page number 5
Getting data about zobacz-wiecej-polityka-obronna from page number 6
Getting data about zobacz-wiecej-polityka-obronna from page number 7
Getting data about zobacz-wiecej-polityka-obronna from page number 8
Getting data about zobacz-wiecej-polityka-obronna from page number 9
Getting data about zobacz-wiecej-przemysl from page number 1
Getting data about zobacz-wiecej-przemysl from page number 2
Getting data about zobacz-wiecej-przemysl from page number 3
Getting data about zobacz-wiecej-przemysl from page number 4
Getting data about zobacz-wiecej-przemysl from page number 5
Getting data about zobacz-wiecej-przemysl from page number 6
Getting data 

In [9]:
df1 = pd.DataFrame(lista[0])
df2 = pd.DataFrame(lista[1])
df3 = pd.DataFrame(lista[2])
df4 = pd.DataFrame(lista[3])

In [10]:
nlp = spacy.load('pl_core_news_sm') 

In [11]:
df_all = pd.concat([df1,df2,df3,df4]).sample(frac=1).reset_index(drop=True)

title_list_lem = []
for title in df_all["title"]:
  doc = nlp(title)
  lista1 = []
  for token in doc:
    lista1.append(token.lemma_)
  title_list_lem.append(" ".join(lista1))

In [12]:
title_list_stop = []
for title in df_all["title"]:
  doc = nlp(title)
  lista1 = []
  for token in doc:
    if token.is_stop == False:
      lista1.append(token.lemma_)
  title_list_stop.append(" ".join(lista1))

In [ ]:
df_all["token_stop"] = title_list_stop
df_all["token_lem"] = title_list_lem